In [1]:
import torch
from torch import nn
import random,numpy
import numpy as np
from copy import deepcopy
from game import Game, Move, RandomPlayer, MyPlayer

In [2]:
class QuixoNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5*5, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 44),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

## Training the agent

In [3]:
## Constants definition
num_iterations = 50_000
num_matches = 1
max_dim_replay_buff = 10_000
time_to_update = 100
gamma = 0.001

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [4]:
class ReplayBuffer:    
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size        
        self.buffer = []
        self.position = 0
        
    def add_experience(self, experience):        
        if len(self.buffer) < self.buffer_size:
            self.buffer.append(experience)        
        else:
            self.buffer[self.position] = experience        
            self.position = (self.position + 1) % self.buffer_size
            
    def sample_batch(self, batch_size):
        """
        #batch = np.random.choice(self.buffer, batch_size, replace=False)        
        #states, actions, rewards, next_states, dones = zip(*batch)
        
        ### OCCHIO QUAAAA !!!!! ###
        
        return (np.array(states),
                np.array(actions),            
                np.array(rewards),
                np.array(next_states),            
                np.array(dones)
        )
        """
        return np.random.sample(self.buffer, batch_size)     

In [5]:
from game import Player


def training(player1: 'Player', player2: 'Player'):
    loss_tot = []

    replay_buff = ReplayBuffer(max_dim_replay_buff) # replay buffer, from which we sample for BATCH learning
    # inizializza i modelli -> passali per parametro
    
    for step in range(num_iterations):        
        # we fill the replay buffer with experiences made with matches
        for match in range(num_matches):
            # lo uso solo in inferenza
            # gioca una intera partita qui dentro e per ogni mossa, metti l'experience dentro il ReplayBuffer
            # ...
            # ... qui descrivere come viene effettivamente usata la rete neurale (magari impacchettarla in qualche funzione)
            winner = -1
            g = Game()
            #player1 = MyPlayer()
            #player2 = RandomPlayer()
            players = [player1, player2]
            
            while winner < 0:
                print("### miao ###")
                g.current_player_idx += 1
                g.current_player_idx %= len(players)
                #print(g.current_player_idx)
                prev_state=deepcopy(g)
                from_pos, slide = players[g.current_player_idx].make_move(g)
                   
                if g.current_player_idx==0:
                    ##l'azione è il valore del q-value quindi un numero? o la posizione in questo caso  
                    ##in quel caso replay_row=(prev_state.get_board,GeneratorNet((prev_state.get_board),g.get_board,reward)
                    reward=g.compute_reward(from_pos, slide)
                    replay_row=(prev_state.get_board, player1.last_action_value, deepcopy(g), reward)
                    replay_buff.add_experience(replay_row)
                    
                if g.check_winner() != -1:
                    break
            
            # replay_buffer.push(...) per ogni transizione di stato osservata ()
            # restituisci il risultato sotto forma di tupla (st, a, st+1, r) - (State, action, next_state, reward)
            
        # Now we sample a batch of data from the ReplayBuffer in order to train the Agent
        batch_to_train = replay_buff.sample_batch(30)
        TargetNet_targets = []
        GeneratorNet_outputs = []
        player1.myplayer_zero_grad()
        # Now we need to compute the list of targets (made with the TargetNet) and the one with the Q values for the "current" state
        
        for element in batch_to_train:
            _, action_val, new_state, reward = element
            print("### {action_val} ###")
            # per ogni tupla di 4 elementi prendere l'action (a) (sarà un valore...q value)
            # Rappresenta il primo termine della Loss function (MSE) => output
            # inserirlo dentro GeneratorNet_outputs
            GeneratorNet_outputs.append(action_val)

            # Per calcolare il secondo termine, devo prendere la reward dalla tupla, gamma dai parametri (vedi sopra) e il q value dell'azione migliore dello stato successivo
            # Rappresenta il secondo termine della Loss function (MSE) => target
            # Per calcolare il target, dare in pasto alla NN lo stato+1 contenuto nella tupla element
            # inserirlo dentro TargetNet_targets
            max_action_newstate = player1.compute_target(new_state)
            res = reward + gamma*max_action_newstate
            TargetNet_targets.append(res)
            
        loss_curr = player1.myplayer_loss_and_update_params(GeneratorNet_outputs, TargetNet_targets) 
        loss_tot.append(loss_curr)
        
        if (step % time_to_update) == 0:
            # update the parameter of the TargetNet
            player1.copy_params_TargetNet()
            
        print(loss_curr)
        #printa come varia 
            
    return player1.GeneratorNet.state_dict()
            
      

In [6]:
player1 = MyPlayer()
player2 = RandomPlayer()
trained_model_params = training(player1, player2)
TrainedGeneratorNet = QuixoNet.load_state_dict(trained_model_params) # TRAINED NETWORK to use in INFERENCE PHASE

### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###
### miao ###


TypeError: cannot unpack non-iterable numpy.float32 object

## Inference

In [ ]:
wins = 0
losts = 0
draws = 0
num_match_test = 1000

for _ in num_match_test:
    # play_match(TrainedGeneratedNet, random)
    # increment wins/losts/draws 
    print("miao")

print(f"Accuracy: {(wins/num_match_test)*100}")
print("Wins: {wins} - Losts: {losts} - Draws {draws}")

